<a href="https://colab.research.google.com/github/Andresknaz/Andresknaz/blob/main/Sistema_Experto_CLIPSPY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -U clipspy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 901.2/901.2 kB 23.6 MB/s eta 0:00:00


In [8]:
KB_TEXT = r"""
; =========================
; PLANTILLAS
; =========================
(deftemplate wifi-case
  (slot slow)         ; yes/no
  (slot drops)        ; yes/no
  (slot far)          ; yes/no
  (slot many-devices) ; yes/no
  (slot old-router)   ; yes/no
)

(deftemplate diagnosis
  (slot problem)
  (slot recommendation)
)

; =========================
; REGLAS (>=3)
; =========================

(defrule r-weak-signal
  (wifi-case (slow yes) (far yes))
  =>
  (assert (diagnosis
    (problem "Senal debil / baja cobertura")
    (recommendation "Acercate al router, ubicalo en un punto alto/centrado o usa repetidor/mesh.")
  ))
)

(defrule r-old-router-drops
  (wifi-case (drops yes) (old-router yes))
  =>
  (assert (diagnosis
    (problem "Router obsoleto o inestable")
    (recommendation "Reinicia, actualiza firmware y considera cambiar a un router WiFi 5/6.")
  ))
)

(defrule r-congestion
  (wifi-case (slow yes) (many-devices yes))
  =>
  (assert (diagnosis
    (problem "Congestion por muchos dispositivos")
    (recommendation "Desconecta equipos inactivos, usa 5GHz si puedes y configura QoS.")
  ))
)

(defrule r-drops-far
  (wifi-case (drops yes) (far yes))
  =>
  (assert (diagnosis
    (problem "Cortes por distancia/interferencias")
    (recommendation "Cambia canal WiFi, reduce interferencias, o instala mesh/repetidor.")
  ))
)

(defrule r-no-issue
  (wifi-case (slow no) (drops no))
  =>
  (assert (diagnosis
    (problem "Sin sintomas claros")
    (recommendation "Haz test de velocidad/latencia y revisa si el problema es del proveedor.")
  ))
)
"""

with open("wifi_kb.clp", "w", encoding="utf-8") as f:
    f.write(KB_TEXT)

print("Archivo creado: wifi_kb.clp")

Archivo creado: wifi_kb.clp


In [9]:
from clips import Environment

def run_expert_system(slow, drops, far, many_devices, old_router):
    env = Environment()

    # Cargar el archivo .clp
    env.load("wifi_kb.clp")
    env.reset()

    fact = f'(wifi-case (slow {slow}) (drops {drops}) (far {far}) (many-devices {many_devices}) (old-router {old_router}))'
    env.assert_string(fact)

    env.run()

    results = []
    for f in env.facts():
        if f.template.name == "diagnosis":
            results.append((f["problem"], f["recommendation"]))
    return results

# ---- Caso de prueba ----
inputs = dict(
    slow="yes",
    drops="no",
    far="yes",
    many_devices="yes",
    old_router="no"
)

print("ENTRADAS:", inputs)
print("\n=== RESULTADOS ===")
for i, (problem, rec) in enumerate(run_expert_system(**inputs), 1):
    print(f"{i}. Problema: {problem}")
    print(f"   Recomendacion: {rec}\n")

ENTRADAS: {'slow': 'yes', 'drops': 'no', 'far': 'yes', 'many_devices': 'yes', 'old_router': 'no'}

=== RESULTADOS ===
1. Problema: Senal debil / baja cobertura
   Recomendacion: Acercate al router, ubicalo en un punto alto/centrado o usa repetidor/mesh.

2. Problema: Congestion por muchos dispositivos
   Recomendacion: Desconecta equipos inactivos, usa 5GHz si puedes y configura QoS.

